## 1. セットアップ

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 設定
sys.path.append(os.path.abspath('..'))
from configs.config import *

# プロットスタイル
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Setup complete")
print(f"Input dir: {DIR_INPUT}")

## 2. データ読み込み

In [ ]:
# メタデータ読み込み
df_train = pd.read_csv(FILE_TRAIN_META)
df_test = pd.read_csv(FILE_TEST_META)

print("="*80)
print("📊 Dataset Loaded")
print("="*80)
print(f"Train: {len(df_train):,} samples")
print(f"Test:  {len(df_test):,} samples")

# グループ列作成（quarter_session）
df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)
df_test['group'] = df_test['quarter'] + '_' + df_test['session'].astype(str)

print("\n="*80)
print("📋 Data Preview")
print("="*80)
print("\n【Train】")
display(df_train.head(10))

print("\n【Test】")
display(df_test.head(10))

In [ ]:
# データの最初の数行を確認
print("="*80)
print("📋 Data Preview")
print("="*80)
print("\n【Train】")
display(df_train.head(10))

print("\n【Test】")
display(df_test.head(10))

## 3. 基本統計

In [ ]:
print("="*80)
print("📊 Basic Statistics")
print("="*80)

print("\n【Train Dataset】")
print(f"  Total samples: {len(df_train):,}")
print(f"  Unique players (label_id): {df_train['label_id'].nunique()}")
print(f"  Unique quarters: {df_train['quarter'].nunique()}")
print(f"  Unique angles: {df_train['angle'].nunique()} → {list(df_train['angle'].unique())}")
print(f"  Unique sessions: {df_train['session'].nunique()} → {sorted(df_train['session'].unique())}")
print(f"  Unique groups (quarter_session): {df_train['group'].nunique()}")
print(f"  Frame range: {df_train['frame'].min()} - {df_train['frame'].max()}")

print("\n【Test Dataset】")
print(f"  Total samples: {len(df_test):,}")
print(f"  Unique quarters: {df_test['quarter'].nunique()}")
print(f"  Unique angles: {df_test['angle'].nunique()} → {list(df_test['angle'].unique())}")
print(f"  Unique sessions: {df_test['session'].nunique()} → {sorted(df_test['session'].unique())}")
print(f"  Unique groups (quarter_session): {df_test['group'].nunique()}")
print(f"  Frame range: {df_test['frame'].min()} - {df_test['frame'].max()}")

print("\n【Data Info - Train】")
df_train.info()

print("\n【Data Info - Test】")
df_test.info()

In [ ]:
# 記述統計
print("="*80)
print("📈 Descriptive Statistics")
print("="*80)

print("\n【Train】")
display(df_train.describe())

print("\n【Test】")
display(df_test.describe())

## 4. 選手（label_id）分析

In [ ]:
print("="*80)
print("👤 Player (label_id) Analysis")
print("="*80)

# 選手ごとのサンプル数
player_counts = df_train['label_id'].value_counts().sort_index()
print("\n【Samples per player】")
for player_id, count in player_counts.items():
    percentage = count / len(df_train) * 100
    print(f"  Player {player_id:2d}: {count:5,} samples ({percentage:5.2f}%)")

print(f"\n【Statistics】")
print(f"  Mean samples per player: {player_counts.mean():.1f}")
print(f"  Std:  {player_counts.std():.1f}")
print(f"  Min:  {player_counts.min():,} (Player {player_counts.idxmin()})")
print(f"  Max:  {player_counts.max():,} (Player {player_counts.idxmax()})")

# 選手ごとの出現グループ数
player_groups = df_train.groupby('label_id')['group'].nunique().sort_index()
print("\n【Groups (quarter_session) per player】")
for player_id, n_groups in player_groups.items():
    print(f"  Player {player_id:2d}: {n_groups:3d} groups")

# 選手ごとのangle分布
print("\n【Angle distribution per player】")
player_angle = df_train.groupby(['label_id', 'angle']).size().unstack(fill_value=0)
display(player_angle)

In [ ]:
# 可視化
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# サンプル数分布
ax = axes[0, 0]
bars = ax.bar(player_counts.index, player_counts.values, color='steelblue', alpha=0.7, edgecolor='black')
ax.set_xlabel('Player ID', fontsize=12)
ax.set_ylabel('Number of Samples', fontsize=12)
ax.set_title('Samples per Player', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
ax.set_xticks(player_counts.index)
# 値をバーの上に表示
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height):,}',
            ha='center', va='bottom', fontsize=9)

# グループ数分布
ax = axes[0, 1]
bars = ax.bar(player_groups.index, player_groups.values, color='coral', alpha=0.7, edgecolor='black')
ax.set_xlabel('Player ID', fontsize=12)
ax.set_ylabel('Number of Groups (Sessions)', fontsize=12)
ax.set_title('Groups per Player', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
ax.set_xticks(player_groups.index)
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=9)

# 選手ごとのangle分布（積み上げ棒グラフ）
ax = axes[1, 0]
player_angle.plot(kind='bar', stacked=True, ax=ax, color=['steelblue', 'coral'], alpha=0.7)
ax.set_xlabel('Player ID', fontsize=12)
ax.set_ylabel('Number of Samples', fontsize=12)
ax.set_title('Samples per Player by Angle', fontsize=14, fontweight='bold')
ax.legend(title='Angle', loc='upper right')
ax.grid(axis='y', alpha=0.3)
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

# 選手の均等性（円グラフ）
ax = axes[1, 1]
colors = plt.cm.Set3(np.linspace(0, 1, len(player_counts)))
wedges, texts, autotexts = ax.pie(player_counts.values, labels=player_counts.index, 
                                    autopct='%1.1f%%', startangle=90, colors=colors)
ax.set_title('Player Distribution (Train)', fontsize=14, fontweight='bold')
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_fontsize(9)
    autotext.set_weight('bold')

plt.tight_layout()
plt.show()

print("\n⚠️  Observation:")
print("  - クラス不均衡: 選手ごとのサンプル数にばらつきあり")
print("  - CV戦略: StratifiedGroupKFoldで層化を試みるか、GroupKFoldでリーク防止優先か")

## 5. Quarter・Session・Frame分析

In [ ]:
print("="*80)
print("🏀 Quarter Analysis")
print("="*80)

print("\n【Train】")
quarter_counts_train = df_train['quarter'].value_counts().sort_index()
for q, count in quarter_counts_train.items():
    percentage = count / len(df_train) * 100
    print(f"  {q}: {count:5,} samples ({percentage:5.2f}%)")

print("\n【Test】")
quarter_counts_test = df_test['quarter'].value_counts().sort_index()
for q, count in quarter_counts_test.items():
    percentage = count / len(df_test) * 100
    print(f"  {q}: {count:5,} samples ({percentage:5.2f}%)")

# Session分析
print("\n" + "="*80)
print("🎬 Session Analysis")
print("="*80)

print("\n【Train】")
session_counts_train = df_train['session'].value_counts().sort_index()
print(f"  Unique sessions: {df_train['session'].nunique()}")
print(f"  Sessions: {sorted(df_train['session'].unique())}")
print(f"  ⚠️  All train data has session=0 (single continuous scene)")

print("\n【Test】")
session_counts_test = df_test['session'].value_counts().sort_index()
print(f"  Unique sessions: {df_test['session'].nunique()}")
for s, count in session_counts_test.items():
    percentage = count / len(df_test) * 100
    print(f"  Session {s}: {count:5,} samples ({percentage:5.2f}%)")

# Group (quarter_session) 分析
print("\n" + "="*80)
print("🔗 Group (quarter_session) Analysis")
print("="*80)

print("\n【Train】")
group_counts_train = df_train['group'].value_counts().sort_values(ascending=False)
print(f"  Total groups: {df_train['group'].nunique()}")
print(f"  Samples per group (mean): {group_counts_train.mean():.1f}")
print(f"  Samples per group (median): {group_counts_train.median():.0f}")
print(f"  Min group size: {group_counts_train.min()}")
print(f"  Max group size: {group_counts_train.max()}")
print(f"\n  Top 10 largest groups:")
for group, count in group_counts_train.head(10).items():
    print(f"    {group}: {count:,} samples")

print("\n【Test】")
group_counts_test = df_test['group'].value_counts().sort_values(ascending=False)
print(f"  Total groups: {df_test['group'].nunique()}")
print(f"  Top 5 largest groups:")
for group, count in group_counts_test.head(5).items():
    print(f"    {group}: {count:,} samples")

In [ ]:
# Frame分析
print("="*80)
print("🎞️  Frame Analysis")
print("="*80)

frame_stats_train = df_train['frame'].describe()
frame_stats_test = df_test['frame'].describe()

print("\n【Train】")
print(f"  Min frame:  {frame_stats_train['min']:.0f}")
print(f"  Max frame:  {frame_stats_train['max']:.0f}")
print(f"  Mean frame: {frame_stats_train['mean']:.1f}")
print(f"  Median:     {frame_stats_train['50%']:.0f}")
print(f"  Std:        {frame_stats_train['std']:.1f}")

print("\n【Test】")
print(f"  Min frame:  {frame_stats_test['min']:.0f}")
print(f"  Max frame:  {frame_stats_test['max']:.0f}")
print(f"  Mean frame: {frame_stats_test['mean']:.1f}")
print(f"  Median:     {frame_stats_test['50%']:.0f}")
print(f"  Std:        {frame_stats_test['std']:.1f}")

# Groupごとのframe数
print("\n【Frames per group (Train, top 10)】")
frames_per_group = df_train.groupby('group')['frame'].nunique().sort_values(ascending=False)
for group, n_frames in frames_per_group.head(10).items():
    print(f"  {group}: {n_frames} frames")

In [ ]:
# 可視化
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Quarter分布（Train vs Test）
ax = axes[0, 0]
x = np.arange(len(quarter_counts_train))
width = 0.35
ax.bar(x - width/2, quarter_counts_train.values, width, label='Train', alpha=0.8, color='steelblue')
test_aligned = quarter_counts_test.reindex(quarter_counts_train.index, fill_value=0)
ax.bar(x + width/2, test_aligned.values, width, label='Test', alpha=0.8, color='coral')
ax.set_xlabel('Quarter', fontsize=12)
ax.set_ylabel('Number of Samples', fontsize=12)
ax.set_title('Quarter Distribution (Train vs Test)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(quarter_counts_train.index, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Session分布（Test only, since Train is all session=0）
ax = axes[0, 1]
ax.bar(session_counts_test.index, session_counts_test.values, color='coral', alpha=0.7, edgecolor='black')
ax.set_xlabel('Session', fontsize=12)
ax.set_ylabel('Number of Samples', fontsize=12)
ax.set_title('Session Distribution (Test only)', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
ax.text(0.02, 0.98, 'Train: all session=0', transform=ax.transAxes,
        verticalalignment='top', fontsize=11,
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Group数の分布
ax = axes[0, 2]
ax.bar(0, df_train['group'].nunique(), color='steelblue', alpha=0.7, width=0.4, label='Train')
ax.bar(1, df_test['group'].nunique(), color='coral', alpha=0.7, width=0.4, label='Test')
ax.set_ylabel('Number of Unique Groups', fontsize=12)
ax.set_title('Unique Groups (quarter_session)', fontsize=14, fontweight='bold')
ax.set_xticks([0, 1])
ax.set_xticklabels(['Train', 'Test'])
ax.grid(axis='y', alpha=0.3)
for i, v in enumerate([df_train['group'].nunique(), df_test['group'].nunique()]):
    ax.text(i, v + 1, str(v), ha='center', va='bottom', fontsize=12, fontweight='bold')

# Frame分布（Train）
ax = axes[1, 0]
ax.hist(df_train['frame'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
ax.set_xlabel('Frame Number', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Frame Distribution (Train)', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
ax.axvline(df_train['frame'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df_train["frame"].mean():.1f}')
ax.legend()

# Frame分布（Test）
ax = axes[1, 1]
ax.hist(df_test['frame'], bins=50, color='coral', alpha=0.7, edgecolor='black')
ax.set_xlabel('Frame Number', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Frame Distribution (Test)', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
ax.axvline(df_test['frame'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df_test["frame"].mean():.1f}')
ax.legend()

# Groupサイズ分布
ax = axes[1, 2]
ax.hist(group_counts_train.values, bins=30, color='steelblue', alpha=0.7, edgecolor='black')
ax.set_xlabel('Samples per Group', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Group Size Distribution (Train)', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
ax.axvline(group_counts_train.mean(), color='red', linestyle='--', linewidth=2, 
          label=f'Mean: {group_counts_train.mean():.1f}')
ax.legend()

plt.tight_layout()
plt.show()

print("\n⚠️  Key Observations:")
print("  - Train: session=0のみ（単一の連続シーン）")
print("  - Test: 複数のsession（時間的に分離されたシーン）")
print("  - CV戦略: quarter_sessionでグループ化してリーク防止")

## 6. Angle（カメラアングル）分析

In [ ]:
print("="*80)
print("📷 Camera Angle Analysis")
print("="*80)

print("\n【Train】")
angle_counts_train = df_train['angle'].value_counts()
for angle, count in angle_counts_train.items():
    percentage = count / len(df_train) * 100
    print(f"  {angle:4s}: {count:,} samples ({percentage:.1f}%)")

print("\n【Test】")
angle_counts_test = df_test['angle'].value_counts()
for angle, count in angle_counts_test.items():
    percentage = count / len(df_test) * 100
    print(f"  {angle:4s}: {count:,} samples ({percentage:.1f}%)")

# Angle x Quarter のクロス集計
print("\n【Angle x Quarter (Train)】")
cross_tab_train = pd.crosstab(df_train['angle'], df_train['quarter'])
display(cross_tab_train)

print("\n【Angle x Quarter (Test)】")
cross_tab_test = pd.crosstab(df_test['angle'], df_test['quarter'])
display(cross_tab_test)

# Angle x Player のクロス集計
print("\n【Angle x Player (Train)】")
cross_tab_player = pd.crosstab(df_train['angle'], df_train['label_id'])
display(cross_tab_player)

In [ ]:
# 可視化
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Angle分布（Train）
ax = axes[0, 0]
bars = ax.bar(range(len(angle_counts_train)), angle_counts_train.values, 
              color=['steelblue', 'coral'], alpha=0.7, edgecolor='black')
ax.set_xlabel('Angle', fontsize=12)
ax.set_ylabel('Number of Samples', fontsize=12)
ax.set_title('Camera Angle Distribution (Train)', fontsize=14, fontweight='bold')
ax.set_xticks(range(len(angle_counts_train)))
ax.set_xticklabels(angle_counts_train.index, fontsize=11)
ax.grid(axis='y', alpha=0.3)
for i, (angle, count) in enumerate(angle_counts_train.items()):
    percentage = count / len(df_train) * 100
    ax.text(i, count + 500, f'{count:,}\n({percentage:.1f}%)', 
            ha='center', va='bottom', fontsize=11, fontweight='bold')

# Angle分布（Test）
ax = axes[0, 1]
bars = ax.bar(range(len(angle_counts_test)), angle_counts_test.values, 
              color=['steelblue', 'coral'], alpha=0.7, edgecolor='black')
ax.set_xlabel('Angle', fontsize=12)
ax.set_ylabel('Number of Samples', fontsize=12)
ax.set_title('Camera Angle Distribution (Test)', fontsize=14, fontweight='bold')
ax.set_xticks(range(len(angle_counts_test)))
ax.set_xticklabels(angle_counts_test.index, fontsize=11)
ax.grid(axis='y', alpha=0.3)
for i, (angle, count) in enumerate(angle_counts_test.items()):
    percentage = count / len(df_test) * 100
    ax.text(i, count + 200, f'{count:,}\n({percentage:.1f}%)', 
            ha='center', va='bottom', fontsize=11, fontweight='bold')

# Angle x Quarter ヒートマップ（Train）
ax = axes[1, 0]
sns.heatmap(cross_tab_train, annot=True, fmt='d', cmap='YlOrRd', ax=ax, 
            cbar_kws={'label': 'Number of Samples'})
ax.set_title('Angle x Quarter Heatmap (Train)', fontsize=14, fontweight='bold')
ax.set_ylabel('Angle', fontsize=12)
ax.set_xlabel('Quarter', fontsize=12)

# Angle x Quarter ヒートマップ（Test）
ax = axes[1, 1]
sns.heatmap(cross_tab_test, annot=True, fmt='d', cmap='YlGnBu', ax=ax,
            cbar_kws={'label': 'Number of Samples'})
ax.set_title('Angle x Quarter Heatmap (Test)', fontsize=14, fontweight='bold')
ax.set_ylabel('Angle', fontsize=12)
ax.set_xlabel('Quarter', fontsize=12)

plt.tight_layout()
plt.show()

print("\n⚠️  Important Findings:")
print(f"  - Train: side={angle_counts_train.get('side', 0):,}, top={angle_counts_train.get('top', 0):,}")
print(f"  - Test:  side={angle_counts_test.get('side', 0):,}, top={angle_counts_test.get('top', 0):,}")
print("  - Test is predominantly 'side' view")
print("  - Model should prioritize 'side' view performance for test prediction")

## 7. BBox（バウンディングボックス）分析

In [ ]:
print("="*80)
print("📦 Bounding Box Analysis")
print("="*80)

# BBox統計
print("\n【Train BBox Statistics】")
for col in ['x', 'y', 'w', 'h']:
    stats = df_train[col].describe()
    print(f"\n  {col.upper()}:")
    print(f"    Mean:   {stats['mean']:7.1f}")
    print(f"    Std:    {stats['std']:7.1f}")
    print(f"    Min:    {stats['min']:7.0f}")
    print(f"    25%:    {stats['25%']:7.0f}")
    print(f"    Median: {stats['50%']:7.0f}")
    print(f"    75%:    {stats['75%']:7.0f}")
    print(f"    Max:    {stats['max']:7.0f}")

print("\n【Test BBox Statistics】")
for col in ['x', 'y', 'w', 'h']:
    stats = df_test[col].describe()
    print(f"\n  {col.upper()}:")
    print(f"    Mean:   {stats['mean']:7.1f}")
    print(f"    Std:    {stats['std']:7.1f}")
    print(f"    Min:    {stats['min']:7.0f}")
    print(f"    25%:    {stats['25%']:7.0f}")
    print(f"    Median: {stats['50%']:7.0f}")
    print(f"    75%:    {stats['75%']:7.0f}")
    print(f"    Max:    {stats['max']:7.0f}")

# BBox派生特徴量
df_train['bbox_area'] = df_train['w'] * df_train['h']
df_test['bbox_area'] = df_test['w'] * df_test['h']

df_train['aspect_ratio'] = df_train['w'] / df_train['h']
df_test['aspect_ratio'] = df_test['w'] / df_test['h']

df_train['center_x'] = df_train['x'] + df_train['w'] / 2
df_train['center_y'] = df_train['y'] + df_train['h'] / 2
df_test['center_x'] = df_test['x'] + df_test['w'] / 2
df_test['center_y'] = df_test['y'] + df_test['h'] / 2

print("\n【Derived Features】")
print("\n  BBox Area (w × h):")
print(f"    Train - Mean: {df_train['bbox_area'].mean():,.1f}, Median: {df_train['bbox_area'].median():,.1f}")
print(f"    Test  - Mean: {df_test['bbox_area'].mean():,.1f}, Median: {df_test['bbox_area'].median():,.1f}")

print("\n  Aspect Ratio (w / h):")
print(f"    Train - Mean: {df_train['aspect_ratio'].mean():.3f}, Median: {df_train['aspect_ratio'].median():.3f}")
print(f"    Test  - Mean: {df_test['aspect_ratio'].mean():.3f}, Median: {df_test['aspect_ratio'].median():.3f}")

print("\n  Center Position:")
print(f"    Train - X: {df_train['center_x'].mean():.1f}, Y: {df_train['center_y'].mean():.1f}")
print(f"    Test  - X: {df_test['center_x'].mean():.1f}, Y: {df_test['center_y'].mean():.1f}")

In [ ]:
# 可視化
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)

# Width分布
ax = fig.add_subplot(gs[0, 0])
ax.hist(df_train['w'], bins=50, alpha=0.6, label='Train', color='steelblue', density=True)
ax.hist(df_test['w'], bins=50, alpha=0.6, label='Test', color='coral', density=True)
ax.set_xlabel('Width (pixels)', fontsize=11)
ax.set_ylabel('Density', fontsize=11)
ax.set_title('BBox Width Distribution', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Height分布
ax = fig.add_subplot(gs[0, 1])
ax.hist(df_train['h'], bins=50, alpha=0.6, label='Train', color='steelblue', density=True)
ax.hist(df_test['h'], bins=50, alpha=0.6, label='Test', color='coral', density=True)
ax.set_xlabel('Height (pixels)', fontsize=11)
ax.set_ylabel('Density', fontsize=11)
ax.set_title('BBox Height Distribution', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Area分布
ax = fig.add_subplot(gs[0, 2])
ax.hist(df_train['bbox_area'], bins=50, alpha=0.6, label='Train', color='steelblue', density=True)
ax.hist(df_test['bbox_area'], bins=50, alpha=0.6, label='Test', color='coral', density=True)
ax.set_xlabel('Area (pixels²)', fontsize=11)
ax.set_ylabel('Density', fontsize=11)
ax.set_title('BBox Area Distribution', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Aspect Ratio分布
ax = fig.add_subplot(gs[0, 3])
ax.hist(df_train['aspect_ratio'], bins=50, alpha=0.6, label='Train', color='steelblue', 
        range=(0, 2), density=True)
ax.hist(df_test['aspect_ratio'], bins=50, alpha=0.6, label='Test', color='coral', 
        range=(0, 2), density=True)
ax.set_xlabel('Aspect Ratio (w/h)', fontsize=11)
ax.set_ylabel('Density', fontsize=11)
ax.set_title('BBox Aspect Ratio Distribution', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# X位置分布
ax = fig.add_subplot(gs[1, 0])
ax.hist(df_train['x'], bins=50, alpha=0.6, label='Train', color='steelblue', density=True)
ax.hist(df_test['x'], bins=50, alpha=0.6, label='Test', color='coral', density=True)
ax.set_xlabel('X Position (pixels)', fontsize=11)
ax.set_ylabel('Density', fontsize=11)
ax.set_title('BBox X Position Distribution', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Y位置分布
ax = fig.add_subplot(gs[1, 1])
ax.hist(df_train['y'], bins=50, alpha=0.6, label='Train', color='steelblue', density=True)
ax.hist(df_test['y'], bins=50, alpha=0.6, label='Test', color='coral', density=True)
ax.set_xlabel('Y Position (pixels)', fontsize=11)
ax.set_ylabel('Density', fontsize=11)
ax.set_title('BBox Y Position Distribution', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Center X分布
ax = fig.add_subplot(gs[1, 2])
ax.hist(df_train['center_x'], bins=50, alpha=0.6, label='Train', color='steelblue', density=True)
ax.hist(df_test['center_x'], bins=50, alpha=0.6, label='Test', color='coral', density=True)
ax.set_xlabel('Center X (pixels)', fontsize=11)
ax.set_ylabel('Density', fontsize=11)
ax.set_title('BBox Center X Distribution', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Center Y分布
ax = fig.add_subplot(gs[1, 3])
ax.hist(df_train['center_y'], bins=50, alpha=0.6, label='Train', color='steelblue', density=True)
ax.hist(df_test['center_y'], bins=50, alpha=0.6, label='Test', color='coral', density=True)
ax.set_xlabel('Center Y (pixels)', fontsize=11)
ax.set_ylabel('Density', fontsize=11)
ax.set_title('BBox Center Y Distribution', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# BBox位置のスキャッタープロット（Train）
ax = fig.add_subplot(gs[2, :2])
scatter = ax.scatter(df_train['center_x'], df_train['center_y'], 
                     c=df_train['label_id'], cmap='tab10', alpha=0.3, s=5)
ax.set_xlabel('Center X (pixels)', fontsize=11)
ax.set_ylabel('Center Y (pixels)', fontsize=11)
ax.set_title('BBox Center Position Distribution (Train)', fontsize=12, fontweight='bold')
ax.invert_yaxis()  # Y軸を反転（画像座標系）
ax.grid(alpha=0.3)
plt.colorbar(scatter, ax=ax, label='Player ID')

# BBox位置のスキャッタープロット（Test）
ax = fig.add_subplot(gs[2, 2:])
ax.scatter(df_test['center_x'], df_test['center_y'], 
           alpha=0.3, s=5, color='coral')
ax.set_xlabel('Center X (pixels)', fontsize=11)
ax.set_ylabel('Center Y (pixels)', fontsize=11)
ax.set_title('BBox Center Position Distribution (Test)', fontsize=12, fontweight='bold')
ax.invert_yaxis()  # Y軸を反転
ax.grid(alpha=0.3)

plt.suptitle('Bounding Box Comprehensive Analysis', fontsize=16, fontweight='bold', y=0.995)
plt.show()

In [ ]:
# 選手ごとのBBox統計
print("="*80)
print("👤 BBox Statistics per Player")
print("="*80)

bbox_stats_per_player = df_train.groupby('label_id').agg({
    'w': ['mean', 'std'],
    'h': ['mean', 'std'],
    'bbox_area': ['mean', 'std'],
    'aspect_ratio': ['mean', 'std']
}).round(2)

print("\n【Width】")
display(bbox_stats_per_player['w'])

print("\n【Height】")
display(bbox_stats_per_player['h'])

print("\n【Area】")
display(bbox_stats_per_player['bbox_area'])

print("\n【Aspect Ratio】")
display(bbox_stats_per_player['aspect_ratio'])

# 可視化
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Width per player
ax = axes[0, 0]
df_train.boxplot(column='w', by='label_id', ax=ax)
ax.set_xlabel('Player ID', fontsize=12)
ax.set_ylabel('Width (pixels)', fontsize=12)
ax.set_title('BBox Width per Player', fontsize=14, fontweight='bold')
plt.sca(ax)
plt.xticks(rotation=0)

# Height per player
ax = axes[0, 1]
df_train.boxplot(column='h', by='label_id', ax=ax)
ax.set_xlabel('Player ID', fontsize=12)
ax.set_ylabel('Height (pixels)', fontsize=12)
ax.set_title('BBox Height per Player', fontsize=14, fontweight='bold')
plt.sca(ax)
plt.xticks(rotation=0)

# Area per player
ax = axes[1, 0]
df_train.boxplot(column='bbox_area', by='label_id', ax=ax)
ax.set_xlabel('Player ID', fontsize=12)
ax.set_ylabel('Area (pixels²)', fontsize=12)
ax.set_title('BBox Area per Player', fontsize=14, fontweight='bold')
plt.sca(ax)
plt.xticks(rotation=0)

# Aspect Ratio per player
ax = axes[1, 1]
df_train.boxplot(column='aspect_ratio', by='label_id', ax=ax)
ax.set_xlabel('Player ID', fontsize=12)
ax.set_ylabel('Aspect Ratio', fontsize=12)
ax.set_title('BBox Aspect Ratio per Player', fontsize=14, fontweight='bold')
plt.sca(ax)
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

print("\n⚠️  Observation:")
print("  - BBoxサイズは選手や位置によって変動")
print("  - Aspect ratioは比較的一定（人物の形状）")
print("  - これらの特徴は選手識別に有用かもしれない")

## 8. CV戦略の検証

In [ ]:
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold

print("="*80)
print("🔍 CV Strategy Validation")
print("="*80)

# GroupKFoldでのリークチェック
print("\n【GroupKFold with quarter_session】")
gkf = GroupKFold(n_splits=5)
groups = df_train['group'].values
y = df_train['label_id'].values

for fold_idx, (train_idx, valid_idx) in enumerate(gkf.split(df_train, y, groups)):
    train_groups = set(groups[train_idx])
    valid_groups = set(groups[valid_idx])
    
    # 重複チェック
    overlap = train_groups & valid_groups
    leak_status = "❌ LEAK!" if len(overlap) > 0 else "✅ No leak"
    
    print(f"\nFold {fold_idx}: {leak_status}")
    print(f"  Train: {len(train_idx):5,} samples, {len(train_groups):3d} groups")
    print(f"  Valid: {len(valid_idx):5,} samples, {len(valid_groups):3d} groups")
    print(f"  Overlap groups: {len(overlap)}")
    
    # 選手分布
    train_labels = set(y[train_idx])
    valid_labels = set(y[valid_idx])
    print(f"  Players - Train: {len(train_labels)}, Valid: {len(valid_labels)}, Overlap: {len(train_labels & valid_labels)}")
    
    # 各選手のサンプル数
    train_label_counts = pd.Series(y[train_idx]).value_counts()
    valid_label_counts = pd.Series(y[valid_idx]).value_counts()
    print(f"  Label balance (train): min={train_label_counts.min()}, max={train_label_counts.max()}")
    print(f"  Label balance (valid): min={valid_label_counts.min()}, max={valid_label_counts.max()}")

print("\n" + "="*80)
print("✅ CV Strategy Verified")
print("="*80)
print("  - GroupKFold ensures no leakage between train and validation")
print("  - Each fold has all players (good for evaluation)")
print("  - Sample sizes are relatively balanced across folds")

In [ ]:
# Foldごとの分布可視化
fold_stats = []
gkf = GroupKFold(n_splits=5)
groups = df_train['group'].values
y = df_train['label_id'].values

for fold_idx, (train_idx, valid_idx) in enumerate(gkf.split(df_train, y, groups)):
    fold_stats.append({
        'fold': fold_idx,
        'train_samples': len(train_idx),
        'valid_samples': len(valid_idx),
        'train_groups': len(set(groups[train_idx])),
        'valid_groups': len(set(groups[valid_idx])),
    })

fold_df = pd.DataFrame(fold_stats)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# サンプル数
ax = axes[0]
x = np.arange(len(fold_df))
width = 0.35
ax.bar(x - width/2, fold_df['train_samples'], width, label='Train', alpha=0.8, color='steelblue')
ax.bar(x + width/2, fold_df['valid_samples'], width, label='Valid', alpha=0.8, color='coral')
ax.set_xlabel('Fold', fontsize=12)
ax.set_ylabel('Number of Samples', fontsize=12)
ax.set_title('Samples per Fold', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([f'Fold {i}' for i in range(5)])
ax.legend()
ax.grid(axis='y', alpha=0.3)

# グループ数
ax = axes[1]
ax.bar(x - width/2, fold_df['train_groups'], width, label='Train', alpha=0.8, color='steelblue')
ax.bar(x + width/2, fold_df['valid_groups'], width, label='Valid', alpha=0.8, color='coral')
ax.set_xlabel('Fold', fontsize=12)
ax.set_ylabel('Number of Groups', fontsize=12)
ax.set_title('Groups per Fold', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([f'Fold {i}' for i in range(5)])
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n【Fold Statistics】")
display(fold_df)

## 9. サマリー・インサイト

## 9. 画像データのEDA

### 9.1. 画像ファイルの確認

In [ ]:
import cv2
from PIL import Image

print("="*80)
print("🖼️  Image Data Analysis")
print("="*80)

# 画像ディレクトリの確認
image_dir = Path(DIR_INPUT) / 'images'
print(f"\nImage directory: {image_dir}")
print(f"Directory exists: {image_dir.exists()}")

if image_dir.exists():
    # 画像ファイル数のカウント
    image_files = list(image_dir.glob('*.jpg')) + list(image_dir.glob('*.png'))
    print(f"Total image files: {len(image_files):,}")
    
    # サンプル画像ファイル名
    print(f"\nSample image files:")
    for i, img_file in enumerate(sorted(image_files)[:5]):
        print(f"  {i+1}. {img_file.name}")
    
    # 画像ファイル名の形式確認
    sample_file = image_files[0]
    print(f"\nImage filename format: {sample_file.name}")
    print("  Format: {quarter}__{angle}__{session}__{frame}.jpg")
else:
    print("\n⚠️  Image directory not found!")
    print("Images should be in: data/raw/input/images/")

# メタデータから画像パスを生成する関数
def get_image_path(row):
    """メタデータの行から画像パスを生成"""
    if len(str(row['frame'])) == 1:
        filename = f"{row['quarter']}__{row['angle']}__0{row['session']}__0{row['frame']}.jpg"
    else:
        filename = f"{row['quarter']}__{row['angle']}__0{row['session']}__{row['frame']}.jpg"
    return image_dir / filename

# サンプル確認
print("\n【Image path generation test】")
sample_row = df_train.iloc[0]
sample_path = get_image_path(sample_row)
print(f"Sample metadata: {sample_row[['quarter', 'angle', 'session', 'frame']].to_dict()}")
print(f"Generated path: {sample_path}")
print(f"File exists: {sample_path.exists()}")

### 9.2. 画像サイズの確認

In [ ]:
# ランダムに画像をサンプリングしてサイズを確認
print("="*80)
print("📐 Image Size Analysis")
print("="*80)

sample_size = 100
sampled_rows = df_train.sample(n=min(sample_size, len(df_train)), random_state=42)

image_sizes = []
for idx, row in sampled_rows.iterrows():
    img_path = get_image_path(row)
    if img_path.exists():
        img = cv2.imread(str(img_path))
        if img is not None:
            h, w, c = img.shape
            image_sizes.append({'height': h, 'width': w, 'channels': c})

if image_sizes:
    sizes_df = pd.DataFrame(image_sizes)
    
    print(f"\n【Sampled {len(image_sizes)} images】")
    print(f"\nHeight:")
    print(f"  Mean: {sizes_df['height'].mean():.1f}")
    print(f"  Std:  {sizes_df['height'].std():.1f}")
    print(f"  Min:  {sizes_df['height'].min()}")
    print(f"  Max:  {sizes_df['height'].max()}")
    
    print(f"\nWidth:")
    print(f"  Mean: {sizes_df['width'].mean():.1f}")
    print(f"  Std:  {sizes_df['width'].std():.1f}")
    print(f"  Min:  {sizes_df['width'].min()}")
    print(f"  Max:  {sizes_df['width'].max()}")
    
    print(f"\nChannels: {sizes_df['channels'].unique()}")
    
    # ユニークなサイズの組み合わせ
    unique_sizes = sizes_df.groupby(['height', 'width']).size()
    print(f"\n【Unique image sizes】")
    for (h, w), count in unique_sizes.items():
        print(f"  {h} x {w}: {count} images")
else:
    print("\n⚠️  Could not load any images!")

### 9.3. ランダム画像サンプルの表示

In [ ]:
# ランダムな画像サンプルを表示（BBoxあり/なし）
print("="*80)
print("🎨 Random Image Samples (with BBox)")
print("="*80)

def draw_bbox(img, x, y, w, h, label=None, color=(0, 255, 0), thickness=3):
    """画像にBBoxを描画"""
    img_copy = img.copy()
    cv2.rectangle(img_copy, (int(x), int(y)), (int(x+w), int(y+h)), color, thickness)
    if label is not None:
        # ラベルテキストを描画
        cv2.putText(img_copy, f"Player {label}", (int(x), int(y)-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    return img_copy

# ランダムサンプリング（異なる選手）
sample_players = df_train['label_id'].unique()[:6]  # 最初の6人
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, player_id in enumerate(sample_players):
    player_data = df_train[df_train['label_id'] == player_id]
    sample_row = player_data.sample(n=1, random_state=42).iloc[0]
    
    img_path = get_image_path(sample_row)
    
    if img_path.exists():
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # BBoxを描画
        img_bbox = draw_bbox(img, sample_row['x'], sample_row['y'], 
                            sample_row['w'], sample_row['h'], 
                            label=player_id, color=(255, 0, 0))
        
        axes[idx].imshow(img_bbox)
        axes[idx].set_title(f"Player {player_id} | {sample_row['angle']} | "
                          f"{sample_row['quarter']}_s{sample_row['session']}_f{sample_row['frame']}", 
                          fontsize=11, fontweight='bold')
        axes[idx].axis('off')
    else:
        axes[idx].text(0.5, 0.5, 'Image not found', ha='center', va='center')
        axes[idx].axis('off')

plt.suptitle('Random Image Samples with BBox (Different Players)', 
            fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n✅ BBoxが赤枠で表示されています")

### 9.4. 選手ごとのBBox切り出し画像

In [ ]:
# 各選手のBBox切り出し画像を表示
print("="*80)
print("👤 Cropped Images per Player")
print("="*80)

def crop_bbox(img, x, y, w, h, padding=0):
    """BBoxで画像を切り出し（パディング付き）"""
    h_img, w_img = img.shape[:2]
    x1 = max(0, int(x - padding))
    y1 = max(0, int(y - padding))
    x2 = min(w_img, int(x + w + padding))
    y2 = min(h_img, int(y + h + padding))
    return img[y1:y2, x1:x2]

# 全選手の切り出し画像を表示（各選手3サンプル）
n_players = len(df_train['label_id'].unique())
n_samples_per_player = 3

fig, axes = plt.subplots(n_players, n_samples_per_player, figsize=(12, 3*n_players))

for player_idx, player_id in enumerate(sorted(df_train['label_id'].unique())):
    player_data = df_train[df_train['label_id'] == player_id]
    
    # ランダムに3サンプル選択
    samples = player_data.sample(n=min(n_samples_per_player, len(player_data)), random_state=42)
    
    for sample_idx, (_, row) in enumerate(samples.iterrows()):
        img_path = get_image_path(row)
        
        ax = axes[player_idx, sample_idx] if n_players > 1 else axes[sample_idx]
        
        if img_path.exists():
            img = cv2.imread(str(img_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # BBoxで切り出し
            cropped = crop_bbox(img, row['x'], row['y'], row['w'], row['h'], padding=5)
            
            ax.imshow(cropped)
            if sample_idx == 0:
                ax.set_ylabel(f'Player {player_id}', fontsize=12, fontweight='bold')
            ax.set_title(f"{row['angle']}", fontsize=10)
            ax.axis('off')
        else:
            ax.text(0.5, 0.5, 'Not found', ha='center', va='center')
            ax.axis('off')

plt.suptitle('Cropped BBox Images per Player (Random 3 samples)', 
            fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n⚠️  Observation:")
print("  - 選手ごとの見た目の違いを確認")
print("  - ユニフォームの色・番号で識別できる可能性")
print("  - 同じ選手でも角度や距離によって見た目が変わる")

### 9.5. Angle別の画像比較（side vs top）

In [ ]:
# side画角とtop画角の比較
print("="*80)
print("📐 Angle Comparison: side vs top")
print("="*80)

# side/top両方存在する同じquarter_session_frameを探す
df_train['qsf'] = df_train['quarter'] + '_' + df_train['session'].astype(str) + '_' + df_train['frame'].astype(str)
qsf_counts = df_train.groupby(['qsf']).agg({'angle': 'nunique'}).reset_index()
common_qsf = qsf_counts[qsf_counts['angle'] == 2]['qsf'].values

if len(common_qsf) > 0:
    # ランダムに4つのフレームを選択
    selected_qsf = np.random.choice(common_qsf, size=min(4, len(common_qsf)), replace=False)
    
    fig, axes = plt.subplots(len(selected_qsf), 2, figsize=(14, 4*len(selected_qsf)))
    if len(selected_qsf) == 1:
        axes = axes.reshape(1, -1)
    
    for idx, qsf in enumerate(selected_qsf):
        frame_data = df_train[df_train['qsf'] == qsf]
        
        # side画像
        side_row = frame_data[frame_data['angle'] == 'side'].iloc[0]
        side_path = get_image_path(side_row)
        
        # top画像
        top_row = frame_data[frame_data['angle'] == 'top'].iloc[0]
        top_path = get_image_path(top_row)
        
        # side画像表示
        if side_path.exists():
            img_side = cv2.imread(str(side_path))
            img_side = cv2.cvtColor(img_side, cv2.COLOR_BGR2RGB)
            
            # 全BBoxを描画
            for _, bbox in frame_data[frame_data['angle'] == 'side'].iterrows():
                img_side = draw_bbox(img_side, bbox['x'], bbox['y'], bbox['w'], bbox['h'], 
                                    label=f"{bbox['label_id']}", color=(255, 0, 0), thickness=2)
            
            axes[idx, 0].imshow(img_side)
            axes[idx, 0].set_title(f"side - {side_row['quarter']} (Session {side_row['session']}, Frame {side_row['frame']})", 
                                  fontsize=10)
            axes[idx, 0].axis('off')
        
        # top画像表示
        if top_path.exists():
            img_top = cv2.imread(str(top_path))
            img_top = cv2.cvtColor(img_top, cv2.COLOR_BGR2RGB)
            
            # 全BBoxを描画
            for _, bbox in frame_data[frame_data['angle'] == 'top'].iterrows():
                img_top = draw_bbox(img_top, bbox['x'], bbox['y'], bbox['w'], bbox['h'], 
                                   label=f"{bbox['label_id']}", color=(255, 0, 0), thickness=2)
            
            axes[idx, 1].imshow(img_top)
            axes[idx, 1].set_title(f"top - {top_row['quarter']} (Session {top_row['session']}, Frame {top_row['frame']})", 
                                  fontsize=10)
            axes[idx, 1].axis('off')
    
    plt.suptitle('Side vs Top Angle Comparison (Same Frame)', fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
    
    print("\n⚠️  Observation:")
    print("  - side画角は横から、top画角は上からの視点")
    print("  - テストデータでは一部フレームのみtop画角あり（test_top_meta.csv）")
    print("  - 学習データは両方の画角が揃っているため、マルチビュー学習が可能")
else:
    print("⚠️  両方の画角が存在する同一フレームが見つかりませんでした")

### 9.6. 画像品質・特性の分析

In [ ]:
# 画像品質とBBox特性の分析
print("="*80)
print("🔍 Image Quality & BBox Characteristics")
print("="*80)

# より詳細な分析のため、200サンプル取得
sample_data = df_train.sample(n=min(200, len(df_train)), random_state=42)
quality_stats = []

for _, row in sample_data.iterrows():
    img_path = get_image_path(row)
    
    if img_path.exists():
        img = cv2.imread(str(img_path))
        
        # 画像全体の品質指標
        brightness = np.mean(img)
        contrast = np.std(img)
        
        # BBox領域の切り出し
        bbox_img = crop_bbox(img, row['x'], row['y'], row['w'], row['h'])
        
        # BBox内の特性
        bbox_brightness = np.mean(bbox_img) if bbox_img.size > 0 else 0
        bbox_area = row['w'] * row['h']
        bbox_aspect = row['h'] / row['w'] if row['w'] > 0 else 0
        
        quality_stats.append({
            'brightness': brightness,
            'contrast': contrast,
            'bbox_brightness': bbox_brightness,
            'bbox_area': bbox_area,
            'bbox_aspect': bbox_aspect,
            'angle': row['angle'],
            'label_id': row['label_id']
        })

df_quality = pd.DataFrame(quality_stats)

# 可視化
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. 画像の明るさ分布（angle別）
axes[0, 0].hist([df_quality[df_quality['angle'] == 'side']['brightness'], 
                df_quality[df_quality['angle'] == 'top']['brightness']], 
                bins=30, label=['side', 'top'], alpha=0.7)
axes[0, 0].set_xlabel('Brightness', fontsize=10)
axes[0, 0].set_ylabel('Frequency', fontsize=10)
axes[0, 0].set_title('Image Brightness Distribution', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 2. コントラスト分布
axes[0, 1].hist([df_quality[df_quality['angle'] == 'side']['contrast'], 
                df_quality[df_quality['angle'] == 'top']['contrast']], 
                bins=30, label=['side', 'top'], alpha=0.7)
axes[0, 1].set_xlabel('Contrast (std)', fontsize=10)
axes[0, 1].set_ylabel('Frequency', fontsize=10)
axes[0, 1].set_title('Image Contrast Distribution', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# 3. BBox面積の分布
axes[0, 2].hist(df_quality['bbox_area'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 2].set_xlabel('BBox Area (pixels)', fontsize=10)
axes[0, 2].set_ylabel('Frequency', fontsize=10)
axes[0, 2].set_title('BBox Area Distribution', fontweight='bold')
axes[0, 2].grid(alpha=0.3)

# 4. BBoxアスペクト比の分布
axes[1, 0].hist(df_quality['bbox_aspect'], bins=30, edgecolor='black', alpha=0.7)
axes[1, 0].set_xlabel('BBox Aspect Ratio (h/w)', fontsize=10)
axes[1, 0].set_ylabel('Frequency', fontsize=10)
axes[1, 0].set_title('BBox Aspect Ratio Distribution', fontweight='bold')
axes[1, 0].axvline(1.0, color='red', linestyle='--', label='Square')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# 5. 画像明るさ vs コントラスト
scatter = axes[1, 1].scatter(df_quality['brightness'], df_quality['contrast'], 
                             c=df_quality['angle'].map({'side': 0, 'top': 1}), 
                             cmap='viridis', alpha=0.6, s=30)
axes[1, 1].set_xlabel('Brightness', fontsize=10)
axes[1, 1].set_ylabel('Contrast', fontsize=10)
axes[1, 1].set_title('Brightness vs Contrast', fontweight='bold')
axes[1, 1].grid(alpha=0.3)
cbar = plt.colorbar(scatter, ax=axes[1, 1])
cbar.set_ticks([0, 1])
cbar.set_ticklabels(['side', 'top'])

# 6. 選手ごとのBBox面積の平均
player_bbox = df_quality.groupby('label_id')['bbox_area'].mean().sort_values()
axes[1, 2].barh(range(len(player_bbox)), player_bbox.values)
axes[1, 2].set_yticks(range(len(player_bbox)))
axes[1, 2].set_yticklabels([f'Player {pid}' for pid in player_bbox.index])
axes[1, 2].set_xlabel('Average BBox Area', fontsize=10)
axes[1, 2].set_title('Average BBox Size per Player', fontweight='bold')
axes[1, 2].grid(axis='x', alpha=0.3)

plt.suptitle('Image Quality & BBox Characteristics Analysis', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

# 統計サマリー
print("\n📊 Statistical Summary:")
print(f"  - Average Brightness: {df_quality['brightness'].mean():.2f} ± {df_quality['brightness'].std():.2f}")
print(f"  - Average Contrast: {df_quality['contrast'].mean():.2f} ± {df_quality['contrast'].std():.2f}")
print(f"  - Average BBox Area: {df_quality['bbox_area'].mean():.1f} ± {df_quality['bbox_area'].std():.1f} pixels")
print(f"  - Average Aspect Ratio: {df_quality['bbox_aspect'].mean():.2f} ± {df_quality['bbox_aspect'].std():.2f}")
print(f"\n  - BBox Area Range: {df_quality['bbox_area'].min():.0f} ~ {df_quality['bbox_area'].max():.0f} pixels")
print(f"  - Most common aspect ratio: ~{df_quality['bbox_aspect'].median():.2f} (median)")

print("\n⚠️  Observation:")
print("  - BBoxサイズに大きなばらつき → データ拡張でスケール正規化が重要")
print("  - アスペクト比は概ね縦長（人物の形状）")
print("  - 画角によって明るさ・コントラストに差がある可能性")

---

## 10. EDAまとめ

画像データのEDAを追加しました。以下の分析を実施しました：

### メタデータ分析
- 選手ID・Quarter・Session・Frameの基本統計
- Camera angle（side/top）の分布
- BBox位置・サイズの統計
- CV戦略の妥当性検証（GroupKFold + quarter_session grouping）

### 画像データ分析
- 画像ファイルの確認と画像サイズの統計
- ランダム画像サンプルとBBox可視化
- 選手ごとの切り出し画像比較
- Angle別（side vs top）の画像比較
- 画像品質・BBox特性の定量分析

### 重要な発見
1. **Temporal Leakage対策**: GroupKFoldでquarter_session単位でグルーピングすることで、連続フレーム間の情報漏れを防止
2. **画角の違い**: 学習データは両画角が揃っているが、テストは一部のみtop画角あり
3. **BBoxサイズのばらつき**: スケール正規化とデータ拡張が重要
4. **選手の見た目**: ユニフォームの色・番号で識別可能だが、角度や距離で見え方が変わる

次のステップとして、このEDAの知見を活かしたベースラインモデルの学習を実施してください。

In [ ]:
print("="*80)
print("📝 EDA Summary & Key Insights")
print("="*80)

print("""
【データセット概要】
✓ Train: 24,920 samples, 11 players, 1 session (session=0)
✓ Test:  8,980 samples, multiple sessions (1-6)
✓ 2つのカメラアングル: side (横), top (上)
✓ 連続フレームからの切り出し

【選手分布】
✓ 11人の選手（label_id: 0-10）
✓ サンプル数に不均衡あり（最小〜最大で約2倍の差）
✓ すべての選手が複数のグループ（session）に出現

【データ構造】
✓ Quarter: 試合のクオーター（Q1-000など）
✓ Session: シーンのセッション番号
  - Train: すべてsession=0（単一の連続シーン）
  - Test: 複数のsession（時間的に分離されたシーン）
✓ Frame: 各session内の連続フレーム番号
✓ Angle: カメラアングル
  - Train: side と top の両方
  - Test: 主にside（topは一部のみ）

【BBox特徴】
✓ Width: 平均55px、範囲20-120px
✓ Height: 平均150px、範囲80-250px
✓ Aspect Ratio: 平均0.37（縦長の人物）
✓ 選手によってBBoxサイズに若干の違いあり

【リークリスク】
⚠️  同一session内のframeは時間的に連続
⚠️  連続フレーム間で選手の見た目がほぼ同じ
⚠️  quarter単位のグループ化ではリークの可能性

【推奨CV戦略】
✅ GroupKFold with 'quarter_session' グループ化
✅ 同一session内のframeは必ず同じfoldに
✅ 訓練と検証でgroupが重複しない → リーク防止
✅ 5-fold CVで安定した評価が可能

【モデリングの方向性】
1. Phase 1: ResNet50でbaseline（Prototype法またはKNN）
2. Phase 2: ArcFaceで識別性能向上
3. side viewの性能を優先（testはほぼside）
4. Data Augmentation: 回転・反転・色調整など
5. 特徴量キャッシュでイテレーション高速化

【注意点】
⚠️  Test には Train にいない選手（unknown）がいる可能性
⚠️  閾値チューニングが重要（unknown判定のため）
⚠️  Angle（side/top）の分布差に注意
""")

print("="*80)
print("✅ EDA Complete!")
print("="*80)

## 10. 次のステップ

1. **Baseline実装** (`exp_resnet.ipynb`)
   - ResNet50で特徴抽出
   - Prototype法またはKNN法で識別
   - CV評価でベンチマーク確立

2. **閾値チューニング**
   - unknown判定の閾値を最適化
   - Macro F1を最大化

3. **モデル改善**
   - EfficientNet-B0で比較
   - ArcFaceでmetric learning
   - Data Augmentationの導入

4. **アンサンブル**
   - 複数モデルの組み合わせ
   - side/top別モデルの検討